## Running End to End experiments!

In this notebook, we:

1. Unpack our dataset (i.e METCLOUD Q&A). We have two!! The first is the 119 questions which we treat as our __test__ set, and the ~2400 dataset which we use for some experiments later on for RAG
2. [OPTIONAL] create a vector database using the selected dataset -> this is for RAG experiments.
3. We use a `QuestionAnswering` class to ask each question in our dataset to our LLM. We have six different LLM's to try; all from Ollama
4. We use a `Verifier` to measure how accurate the responses generated by `QuestionAnswering`; we do this using two different LLMs
5. Fin! We need to plot this which we do in a separate notebook.

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import ollama
from pathlib import Path
from src.qa import QuestionAnswering
from src.vectordb import ChromaDB
from src.verifier import Verifier

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
#this is the path i.e. location to where our datasets are stored on our PC
data_path = Path('/data/')

#now we set our 'experiment' parameters
mode      = 'test'                          #this means we use the test set i.e. 119 questions.
rerank    = False                            #this means we use RERANKING when doing RAG
emb_model = "all-MiniLM-L6-v2"              #this is the RAG embedding model

#now we load the dataset using pandas
if mode == "test":
    data_df = pd.read_csv(data_path/'metcloud-with-id.csv')
else:
    data_df = pd.read_csv(data_path/'METCLOUD_training.csv')

#this just prints how many questions we have in our dataframe
print('data shape:',data_df.shape)

#we now create a vector database using our ChromaDB class that we wrote
#firstly, we create a cache folder to store our embedding model
#then we pass our dataset and chroma will automatically embed the questions
chroma_cache = data_path/f'chroma_cache/chromadb_{mode}_{emb_model}'
chroma_db    = ChromaDB(chroma_cache,
                        data_df,
                        embedding_model = emb_model)

#create directories to store our generated answers.
core_dir = data_path/'Benchmarking'
save_dir = core_dir/f'dataset_{mode}_emb_{emb_model}_rerank_{rerank}'
save_dir.mkdir(exist_ok=True,parents=True)

#now data is the test dataset
data_df =  pd.read_csv(data_path/'metcloud-with-id.csv')

print('mode:',mode)
print('embedding_model:',emb_model)
print('rerank:',rerank)


data shape: (119, 4)


/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


mode: test
embedding_model: all-MiniLM-L6-v2
rerank: False


Quickly check that the chromadb is working as expected. Ask a question, see if we get an appropriate qa pair back

In [19]:
chroma_db.retrieve('what is metcloud',k=1)

["METCLOUD is a secure sovereign cloud service provider that specializes in offering digital modernization through advanced cybersecurity and artificial intelligence. It is designed to support businesses in adopting next-generation technologies for cloud computing and cybersecurity, ensuring that they stay secure, effective, and efficient. METCLOUD's approach is tailored to meet the unique needs of businesses, with a focus on a people-first strategy. The platform is scalable, making it suitable for small to medium-sized enterprises, and it has been recognized for its excellence in the field, including being named the Cybersecurity Firm of the Year by Finance Monthly in the 2021 FinTech Awards."]

### Now we run the Question and Answering loop!
In this cell, we do the following:
1. Define a list of open source models, available on Ollama.
2. Write a for loop to go through each model.
3. 'pull' the model -> this downloads it, if we don't already heave it
4. Creates a `QuestionAnswering` class.
5. Processes the data
6. Asks each question in the dataset and stores the results to our 'save_dir' set earlier. This can use a vector database if we provide it

In [14]:
finetuned = [i['name'] for i in ollama.list()['models'] if 'metcloud' in i['name']]

In [ ]:
#models = ['phi3','mistral','gemma2','llama3','llama3.1','qwen2']
models = finetuned

#loop through each of the models in models
for model in models:
  #download the model if we dont have it  
  #ollama.pull(model)
  print('MODEL:',model)

  #create question/answering class
  qa = QuestionAnswering(model=model)

  #process the dataset
  qa.process_dataset(data_df)

  #ask all questions, saving responses to a .csv file in save_dir
  qa.ask_all_questions(save_dir,
                       vector_db=chroma_db,
                       rerank = rerank)

MODEL: metcloud_1epoch_Qwen2-7B-instruct-bnb-4bit:latest


  1%|          | 1/119 [02:19<4:34:19, 139.49s/it]

False


  2%|▏         | 2/119 [02:21<1:54:17, 58.61s/it] 

False


...

 99%|█████████▉| 118/119 [07:29<00:02,  2.32s/it]

False


100%|██████████| 119/119 [07:32<00:00,  3.81s/it]


False
MODEL: metcloud_1epoch_Phi-3-mini-4k-instruct-bnb-4bit:latest


  0%|          | 0/119 [00:00<?, ?it/s]

In [16]:
!ollama serve

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error: listen tcp 127.0.0.1:11434: bind: address already in use


### Now we do verification i.e. how good were the LLM responses?

1. We grab our answers generated in the previous cell
2. We define a set of 'verification' models. In this case, llama3.1 and gemma2
3. Loop through these models, create a verifier
4. Loop through the generated answers and check if the generated answers were good using verifier
5. Save results

In [9]:
# We first pick which set to verify
rerank = True
mode   = 'test'
emb_model = "all-MiniLM-L6-v2"
data_path = Path('/data/')

#now we define our verifier models i.e vmodels
vmodels = ['llama3.1','gemma2']

#this is the list of tested models
models = ['gemma2','phi3','qwen2','llama3','llama3.1','mistral']

#we loop through our verifier models. This is just two loops
for vmodel in vmodels:
  #makes sure we definitely have the verifier model  
  ollama.pull(vmodel)

  #now we create a verifier class, using our verifier model
  verifier = Verifier(vmodel)

  #'this' is just a directory where we will store our results
  this  = f'dataset_{mode}_emb_{emb_model}_rerank_{rerank}'
  this  = data_path/f'Verification/{this}'
  this.mkdir(exist_ok=True,parents=True)

  #loop through model folders in the question answering directory 
  for model in models:

    #create a directory in our verifier directory to store these results
    pth = this/f'{vmodel}'
    pth.mkdir(exist_ok=True,parents=True)

    #load the csv containing all of the results
    answer_df = pd.read_csv(f'{save_dir}/{model}/all_questions.csv')

    #get verifier to see how good the responses were
    verifier.judge_all_questions(answer_df,model,pth)

100%|██████████| 119/119 [08:04<00:00,  4.07s/it]
